In [2]:
import pandas as pd
import re
from collections import Counter
from operator import itemgetter

In [3]:
def is_url_valid(url):
    if url[:4] != "http":
        return False
    return True
    

def domain_without_www(domain):
    if domain[:4] == "www.":
        return domain[4:]
    return domain


def filter_rows(df):
    df['is_valid'] = df.url.apply(is_url_valid)
    df = df[df['is_valid'] == True]
    df = df.drop(['is_valid'], axis=1)
    df['domain'] = df.domain.apply(domain_without_www)
    df = df.dropna()
    df = df.sort_values('visit_time_ms', ascending=False)
    return df.reset_index().drop('index', axis=1)


def fuck_of_this_domain(df, sites):
    for site in sites:
        df = df[df['domain'] != site]
    return df
    

def clean_date(df):   
    df.day = [re.sub('[-:]', '/', df.day[i]) for i in range(len(df))]
    
    for i in range(len(df)):
        try:
            df.day[i] = pd.to_datetime(df.day[i] + '-'+ df.time[i])
        except ValueError:
            df.day[i] = None
    return filter_rows(df.drop('time', axis=1))


def find_indexes(df, url):
    return df.loc[df['url'] ==  url].index.tolist()


def next_urls(df, url):
    indexes = [element - 1 for element in find_indexes(df, url)]
    return db.url[indexes]


def weighted_urls(df, url, previous_url, NUM_OF_NEXT = 5, NUM_OF_PREVIOUS = 2):
        
    indexes_after = [range(element - NUM_OF_NEXT, element) for element in find_indexes(df, url)]
    indexes_before = [range(element + 1, element + 1 + NUM_OF_PREVIOUS) for element in find_indexes(df, url)]


    weights = [float(1)/2**i for i in range(NUM_OF_NEXT)]
    urls_before = [df.url[i].values  for i in indexes_before]
    urls_after = [df.url[i].values  for i in indexes_after]
    urls_weighted = []
    
    for j in range(len(urls_after)):
        weights = [float(1)/2**(NUM_OF_NEXT - i -1) for i in range(NUM_OF_NEXT)]
        coef = 1
        i = 0
        while i < NUM_OF_PREVIOUS and previous_url[i] == urls_before[j][i]:
            weights = [weight*2 for weight in weights]
            i += 1
        urls_weighted.append([(urls_after[j][k], weights[k]) for k in range(len(weights))])            
    return urls_weighted


def get_weigths(df, url, previous_url):
    weigth_dic = {}
    for url_list in weighted_urls(df, url, previous_url):
        for url in url_list:
            if url[0] in weigth_dic:
                weigth_dic[url[0]] += url[1]
            else:
                weigth_dic[url[0]] = url[1]

    return sorted(weigth_dic.items(), key=itemgetter(1), reverse=True)


def is_in_time(df, index, threshold):
    if df.day[index-1] - df.day[index] < pd.Timedelta(threshold, 's') :
        return False
    else:
        return True

In [30]:
filtered_domains = (['drive.google.com','google.co.il', 'localhost', 'facebook.com', 'gmail.com', 'docs.google.com', 'www.google.co.il', 'messenger.com'])
antho = pd.read_csv('antho_secrets.csv', header=None, delimiter=";")
antho.columns = (['url', 'domain', 'root domain', 'visit_time_ms', 'visit_time_str', 'day of the week', 'transition_type', 'page title'])
df = antho[['url', 'domain', 'visit_time_ms', 'visit_time_str', 'transition_type']]

In [33]:
df = filter_rows(df)
df = fuck_of_this_domain(df, filtered_domains)
k = 100
samples = df['url'][k:k+3].values
for sample in samples:
    print sample

https://docs.python.org/2/library/datetime.html
http://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime
https://docs.python.org/3/library/datetime.html


In [35]:
current_website = samples[0]
previous_websites = samples[1:3]
get_weigths(df, current_website, previous_websites)

[('http://stackoverflow.com/questions/16151402/python-how-can-i-check-whether-an-object-is-of-type-datetime-date',
  4.1875),
 ('https://docs.python.org/3/library/datetime.html', 2.5),
 ('https://docs.python.org/2/library/re.html', 2.0625),
 ('https://docs.python.org/2/library/datetime.html', 1.375),
 ('https://docs.python.org/2.7/tutorial/errors.html', 1.0),
 ('http://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime',
  0.75),
 ('http://stackoverflow.com/questions/14661701/how-to-drop-a-list-of-rows-from-pandas-dataframe',
  0.5),
 ('https://pymotw.com/2/datetime/', 0.5),
 ('http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_timedelta.html',
  0.25),
 ('http://superuser.com/questions/244062/how-do-i-view-add-or-edit-cookies-in-google-chrome',
  0.25),
 ('https://chrome.google.com/webstore/detail/browsing-activity-tracker/maialhkckkpdbhimboiimgdgmhlianje',
  0.125),
 ('https://chrome.google.com/webstore/category/extensions', 0.0625

In [ ]:
df.head(100)